In [1]:
import psycopg2 as ps
import pandas as pd

In [2]:
df = pd.read_csv('csv/video_data_04-11-22.csv')

In [3]:
df.head()

,Id,Title,Published_date,Views,Likes,Comments,Duration
0,V0jlhEAScls,NBA I En İyi Oyunlar,2022-11-04T07:00:11Z,341,14,0,PT1M18S
1,EE9j8Vjslcs,Denver Nuggets - Oklahoma City Thunder Maç Öze...,2022-11-04T06:38:33Z,124,4,0,PT1M36S
2,2Mo1uJpiBl4,Golden State Warriors - Orlando Magic Maç Özet...,2022-11-04T06:38:30Z,393,7,2,PT2M3S
3,vN2q_eycKWw,Olimpia Milano - Real Madrid (77-83) - Maç Öze...,2022-11-03T22:12:26Z,4713,70,8,PT9M9S
4,y2mZrITfk7c,Bayern Münih - Anadolu Efes (81-78) - Maç Özet...,2022-11-03T21:48:40Z,22085,299,99,PT8M19S


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9410 entries, 0 to 9409
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Id              9410 non-null   object
 1   Title           9410 non-null   object
 2   Published_date  9410 non-null   object
 3   Views           9410 non-null   int64 
 4   Likes           9410 non-null   int64 
 5   Comments        9410 non-null   int64 
 6   Duration        9410 non-null   object
dtypes: int64(3), object(4)
memory usage: 514.7+ KB


In [5]:
host_name = 'xxxx'
dbname = 'xxx'
port = 'xxx'
username = 'xxx' 
password = 'xxx'

In [6]:
def connect_to_db(host_name, dbname, port, username, password):
    try:
        conn = ps.connect(host=host_name, database=dbname, user=username, password=password, port=port)

    except ps.OperationalError as e:
        raise e
    else:
        print('Connected!')
        return conn

In [7]:
conn = connect_to_db(host_name, dbname, port, username, password)
curr = conn.cursor()

Connected!


In [8]:
def create_table(curr):
    create_table_command = ("""CREATE TABLE IF NOT EXISTS videos (
                    Id VARCHAR(255) PRIMARY KEY,
                    Title TEXT NOT NULL,
                    Published_date TEXT NOT NULL,
                    Views INTEGER NOT NULL,
                    Likes INTEGER NOT NULL,
                    Comments INTEGER NOT NULL,
                    Duration TEXT NOT NULL
                    
            )""")

    curr.execute(create_table_command)

In [9]:
create_table(curr)

In [10]:
def check_if_video_exists(curr, Id): 
    query = ("""SELECT Id FROM VIDEOS WHERE Id = %s""")

    curr.execute(query, (Id,))
    return curr.fetchone() is not None

def update_row(curr, Id, Title, Published_date, Views, Likes, Comments, Duration):
    query = ("""UPDATE videos
            SET Title = %s,
                Published_date = %s,
                Views = %s,
                Likes = %s,
                Comments = %s,
                Duration = %s
            WHERE Id = %s;""")
    vars_to_update = (Title, Published_date, Views, Likes, Comments, Duration)
    curr.execute(query, vars_to_update)


def update_db(curr,df):
    tmp_df = pd.DataFrame(columns=['Id','Title', 'Published_date', 'Views', 'Likes','Comments','Duration'])
    for i, row in df.iterrows():
        if check_if_video_exists(curr, row['Id']): # If video already exists then we will update
            update_row(curr,row['Id'],row['Title'],row['Published_date'],row['Views'],
                       row['Likes'],row['Comments'],row['Duration'])
        else: # The video doesn't exists so we will add it to a temp df and append it using append_from_df_to_db
            tmp_df = tmp_df.append(row)
            
            #tmp_df = pd.concat([tmp_df, row])

    return tmp_df

In [11]:
#update data for existing videos
new_vid_df = update_db(curr,df)
conn.commit()

C:\Users\kadir\AppData\Local\Temp\ipykernel_2832\1449798468.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tmp_df = tmp_df.append(row)
C:\Users\kadir\AppData\Local\Temp\ipykernel_2832\1449798468.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tmp_df = tmp_df.append(row)
C:\Users\kadir\AppData\Local\Temp\ipykernel_2832\1449798468.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tmp_df = tmp_df.append(row)
C:\Users\kadir\AppData\Local\Temp\ipykernel_2832\1449798468.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tmp_df = tmp_df.append(row)
C:\Users\kadir\AppData\Local\Temp\ipykernel_2832\1449798468.py:27: FutureWarning

C:\Users\kadir\AppData\Local\Temp\ipykernel_2832\1449798468.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tmp_df = tmp_df.append(row)
C:\Users\kadir\AppData\Local\Temp\ipykernel_2832\1449798468.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tmp_df = tmp_df.append(row)
C:\Users\kadir\AppData\Local\Temp\ipykernel_2832\1449798468.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tmp_df = tmp_df.append(row)
C:\Users\kadir\AppData\Local\Temp\ipykernel_2832\1449798468.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tmp_df = tmp_df.append(row)
C:\Users\kadir\AppData\Local\Temp\ipykernel_2832\1449798468.py:27: FutureWarning

C:\Users\kadir\AppData\Local\Temp\ipykernel_2832\1449798468.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tmp_df = tmp_df.append(row)
C:\Users\kadir\AppData\Local\Temp\ipykernel_2832\1449798468.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tmp_df = tmp_df.append(row)
C:\Users\kadir\AppData\Local\Temp\ipykernel_2832\1449798468.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tmp_df = tmp_df.append(row)
C:\Users\kadir\AppData\Local\Temp\ipykernel_2832\1449798468.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tmp_df = tmp_df.append(row)
C:\Users\kadir\AppData\Local\Temp\ipykernel_2832\1449798468.py:27: FutureWarning

C:\Users\kadir\AppData\Local\Temp\ipykernel_2832\1449798468.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tmp_df = tmp_df.append(row)
C:\Users\kadir\AppData\Local\Temp\ipykernel_2832\1449798468.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tmp_df = tmp_df.append(row)
C:\Users\kadir\AppData\Local\Temp\ipykernel_2832\1449798468.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tmp_df = tmp_df.append(row)
C:\Users\kadir\AppData\Local\Temp\ipykernel_2832\1449798468.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tmp_df = tmp_df.append(row)
C:\Users\kadir\AppData\Local\Temp\ipykernel_2832\1449798468.py:27: FutureWarning

C:\Users\kadir\AppData\Local\Temp\ipykernel_2832\1449798468.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tmp_df = tmp_df.append(row)
C:\Users\kadir\AppData\Local\Temp\ipykernel_2832\1449798468.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tmp_df = tmp_df.append(row)
C:\Users\kadir\AppData\Local\Temp\ipykernel_2832\1449798468.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tmp_df = tmp_df.append(row)
C:\Users\kadir\AppData\Local\Temp\ipykernel_2832\1449798468.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tmp_df = tmp_df.append(row)
C:\Users\kadir\AppData\Local\Temp\ipykernel_2832\1449798468.py:27: FutureWarning

...

C:\Users\kadir\AppData\Local\Temp\ipykernel_2832\1449798468.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tmp_df = tmp_df.append(row)
C:\Users\kadir\AppData\Local\Temp\ipykernel_2832\1449798468.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tmp_df = tmp_df.append(row)
C:\Users\kadir\AppData\Local\Temp\ipykernel_2832\1449798468.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tmp_df = tmp_df.append(row)
C:\Users\kadir\AppData\Local\Temp\ipykernel_2832\1449798468.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tmp_df = tmp_df.append(row)
C:\Users\kadir\AppData\Local\Temp\ipykernel_2832\1449798468.py:27: FutureWarning

C:\Users\kadir\AppData\Local\Temp\ipykernel_2832\1449798468.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tmp_df = tmp_df.append(row)
C:\Users\kadir\AppData\Local\Temp\ipykernel_2832\1449798468.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tmp_df = tmp_df.append(row)
C:\Users\kadir\AppData\Local\Temp\ipykernel_2832\1449798468.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tmp_df = tmp_df.append(row)
C:\Users\kadir\AppData\Local\Temp\ipykernel_2832\1449798468.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tmp_df = tmp_df.append(row)
C:\Users\kadir\AppData\Local\Temp\ipykernel_2832\1449798468.py:27: FutureWarning

C:\Users\kadir\AppData\Local\Temp\ipykernel_2832\1449798468.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tmp_df = tmp_df.append(row)
C:\Users\kadir\AppData\Local\Temp\ipykernel_2832\1449798468.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tmp_df = tmp_df.append(row)
C:\Users\kadir\AppData\Local\Temp\ipykernel_2832\1449798468.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tmp_df = tmp_df.append(row)
C:\Users\kadir\AppData\Local\Temp\ipykernel_2832\1449798468.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tmp_df = tmp_df.append(row)
C:\Users\kadir\AppData\Local\Temp\ipykernel_2832\1449798468.py:27: FutureWarning

C:\Users\kadir\AppData\Local\Temp\ipykernel_2832\1449798468.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tmp_df = tmp_df.append(row)
C:\Users\kadir\AppData\Local\Temp\ipykernel_2832\1449798468.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tmp_df = tmp_df.append(row)
C:\Users\kadir\AppData\Local\Temp\ipykernel_2832\1449798468.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tmp_df = tmp_df.append(row)
C:\Users\kadir\AppData\Local\Temp\ipykernel_2832\1449798468.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tmp_df = tmp_df.append(row)
C:\Users\kadir\AppData\Local\Temp\ipykernel_2832\1449798468.py:27: FutureWarning

C:\Users\kadir\AppData\Local\Temp\ipykernel_2832\1449798468.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tmp_df = tmp_df.append(row)
C:\Users\kadir\AppData\Local\Temp\ipykernel_2832\1449798468.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tmp_df = tmp_df.append(row)
C:\Users\kadir\AppData\Local\Temp\ipykernel_2832\1449798468.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tmp_df = tmp_df.append(row)
C:\Users\kadir\AppData\Local\Temp\ipykernel_2832\1449798468.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tmp_df = tmp_df.append(row)
C:\Users\kadir\AppData\Local\Temp\ipykernel_2832\1449798468.py:27: FutureWarning

In [12]:
new_vid_df

,Id,Title,Published_date,Views,Likes,Comments,Duration
0,V0jlhEAScls,NBA I En İyi Oyunlar,2022-11-04T07:00:11Z,341,14,0,PT1M18S
1,EE9j8Vjslcs,Denver Nuggets - Oklahoma City Thunder Maç Öze...,2022-11-04T06:38:33Z,124,4,0,PT1M36S
2,2Mo1uJpiBl4,Golden State Warriors - Orlando Magic Maç Özet...,2022-11-04T06:38:30Z,393,7,2,PT2M3S
3,vN2q_eycKWw,Olimpia Milano - Real Madrid (77-83) - Maç Öze...,2022-11-03T22:12:26Z,4713,70,8,PT9M9S
4,y2mZrITfk7c,Bayern Münih - Anadolu Efes (81-78) - Maç Özet...,2022-11-03T21:48:40Z,22085,299,99,PT8M19S
...,...,...,...,...,...,...,...
9405,IvcdLnQOaJo,Yoel Romero'nun Luke Rockhold'u Nakavt Ettiği ...,2018-03-09T12:17:56Z,1837,21,5,PT34S
9406,_Q_uV0PuPB0,Jessica Andrade - Tecia Torres Dövüşünde Gecen...,2018-03-09T12:17:55Z,1147,12,1,PT25S
9407,btVGCtXxKzg,Marc Marquez'in Şampiyonluk Heyecanı! (MotoGP ...,2018-03-09T12:17:55Z,353,4,4,PT2M7S
9408,xQgGHq76X7Q,Marquez Hata Yaptı Ama... (MotoGP 2017 - Valen...,2018-03-09T12:17:55Z,1105,13,3,PT34S


In [13]:
new_vid_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9410 entries, 0 to 9409
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Id              9410 non-null   object
 1   Title           9410 non-null   object
 2   Published_date  9410 non-null   object
 3   Views           9410 non-null   object
 4   Likes           9410 non-null   object
 5   Comments        9410 non-null   object
 6   Duration        9410 non-null   object
dtypes: object(7)
memory usage: 588.1+ KB


In [16]:
def insert_into_table(curr, Id, Title, Published_date, Views, Likes, Comments, Duration):
    insert_into_videos = ("""INSERT INTO videos (Id, Title, Published_date, Views, Likes, Comments, Duration)
    VALUES(%s,%s,%s,%s,%s,%s,%s);""")
    row_to_insert = (Id, Title, Published_date, Views, Likes, Comments, Duration)
    curr.execute(insert_into_videos, row_to_insert)


def append_from_df_to_db(curr,df):
    for i, row in df.iterrows():
        insert_into_table(curr, row['Id'],row['Title'],row['Published_date'],
                          row['Views'],row['Likes'],row['Comments'],row['Duration'])

In [17]:
append_from_df_to_db(curr, new_vid_df)
conn.commit()

In [18]:
#view data in db table

curr.execute("SELECT * FROM VIDEOS")
print(curr.fetchall())

[('V0jlhEAScls', 'NBA I En İyi Oyunlar', '2022-11-04T07:00:11Z', 341, 14, 0, 'PT1M18S'), ('EE9j8Vjslcs', 'Denver Nuggets - Oklahoma City Thunder Maç Özeti - NBA 2022/23', '2022-11-04T06:38:33Z', 124, 4, 0, 'PT1M36S'), ('2Mo1uJpiBl4', 'Golden State Warriors - Orlando Magic Maç Özeti - NBA 2022/23', '2022-11-04T06:38:30Z', 393, 7, 2, 'PT2M3S'), ('vN2q_eycKWw', 'Olimpia Milano - Real Madrid (77-83) - Maç Özeti - EuroLeague 6. Hafta', '2022-11-03T22:12:26Z', 4713, 70, 8, 'PT9M9S'), ('y2mZrITfk7c', 'Bayern Münih - Anadolu Efes (81-78) - Maç Özeti - EuroLeague 6. Hafta', '2022-11-03T21:48:40Z', 22085, 299, 99, 'PT8M19S'), ('qs2OsoOMRbI', "Alperen Şengün'ün İyi Performansı, Garland İyi Döndü | Miray Çavuşoğlu & İnan Özdemir | NBA Günlüğü", '2022-11-03T21:38:24Z', 2032, 51, 10, 'PT32M16S'), ('914PKEdCrWo', 'Renay Onur ile Özel Röportaj!', '2022-11-03T13:21:03Z', 430, 12, 1, 'PT28M25S'), ('VVyaqfEYwrU', 'Cremonese - Udinese (0-0) - Maç Özeti - Serie A 2022/23', '2022-11-03T11:36:19Z', 398, 22, 

In [32]:
conn.close()

In [13]:
def truncate_table(curr):
    truncate_table = ("""TRUNCATE TABLE videos""")

    curr.execute(truncate_table)

In [14]:
truncate_table(curr)